# SGP4 implementation

This notebook is a step-by-step implementation of the SGP4 algorithm in the heyoka.py expression system, adapted from the ``sgp4.f`` file here:

https://aim.hamptonu.edu/archive/cips/documentation/software/common/astron_lib/

The implementation splits the original code in blocks and compares the numerical values of the quantities computed in each block with numerical values computed directly from the original fortran code.

In [1]:
import heyoka as hy

# Small abs() wrapper.
def ABS(x):
    return hy.select(hy.gte(x, 0.), x, -x)

# ACTAN() wrapper.
def ACTAN(a, b):
    import math
    
    ret = hy.atan2(a, b)
    return hy.select(hy.gte(ret, 0.), ret, 2*math.pi + ret)

# max() wrapper.
def MAX(a, b):
    return hy.select(hy.gt(a, b), a, b)

# min() wrapper.
def MIN(a, b):
    return hy.select(hy.lt(a, b), a, b)

import numpy as np
np.set_printoptions(precision=19)

# Conversion from revolutions per day to
# rad per minute.
def revday2radmin(x):
    import math
    return x*2.*math.pi/1440.

In [2]:
# The inputs.
N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0 = hy.make_vars("N0", "I0", "E0", "BSTAR", "OMEGA0", "M0", "TSINCE", "NODE0")

In [3]:
# Constants.
KE=0.743669161e-1
TOTHRD=2/3.
J2=1.082616e-3
CK2=.5*J2
KMPER=6378.135
S0=20./KMPER
S1=78./KMPER
Q0=120./KMPER
J3=-0.253881e-5
A3OVK2=-J3/CK2
# WHY WHY WHY WHY
J4=float(np.single("-1.65597E-6"))
CK4=-.375*J4
SIMPHT=220./KMPER

In [4]:
# Recover original mean motion (N0DP) and semimajor axis (A0DP)
# from input elements.
A1 = (KE/N0)**TOTHRD
COSI0 = hy.cos(I0)
THETA2 = COSI0**2
X3THM1 = 3.*THETA2-1.
BETA02 = 1.-E0**2
BETA0 = hy.sqrt(BETA02)
DELA2 = 1.5*CK2*X3THM1/(BETA0*BETA02)
DEL1 = DELA2/A1**2
A0 = A1*(1.-DEL1*(1./3.+DEL1*(1.+134./81.*DEL1)))
DEL0 = DELA2/A0**2
N0DP = N0/(1.+DEL0)

In [5]:
cf = hy.cfunc([A1, COSI0, THETA2, X3THM1, BETA02, BETA0, DELA2, DEL1, A0, DEL0, N0DP], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf([revday2radmin(16.05824518), np.deg2rad(72.8435), 0.0086731, .66816e-4, np.deg2rad(52.6988), np.deg2rad(110.5714), 0., np.deg2rad(115.9689)])

cmp_str="1.0405018921296842       0.29498270014673938        8.7014793385861156E-002 -0.73895561984241653       0.99992477733638996       0.99996238796086223       -6.0007159020022667E-004  -5.5426482413560173E-004   1.0406938106368071       -5.5406041460012831E-004   7.0106155666551997E-002"
cmp_arr = np.fromstring(cmp_str, sep=" ")

np.max(np.abs((cmp_arr - res)/cmp_arr))

0.0

In [6]:
# Initialization for new element set.
A0DP = A0/(1.-DEL0)
PERIGE = A0DP*(1.-E0)-1.
S = MIN(MAX(S0,PERIGE-S1),S1)
S4 = 1.+S
PINVSQ = 1./(A0DP*BETA02)**2
XI = 1./(A0DP-S4)
ETA = A0DP*XI*E0
ETASQ = ETA**2
EETA = E0*ETA
PSISQ = ABS(1.-ETASQ)
COEF = ((Q0-S)*XI)**4
COEF1 = COEF/(hy.sqrt(PSISQ)*PSISQ**3)
C1 = BSTAR*COEF1*N0DP*(A0DP*(1.+1.5*ETASQ+EETA*(4.+ETASQ))+0.75*CK2*XI/PSISQ*X3THM1*(8.+3.*ETASQ*(8.+ETASQ)))

In [7]:
cf = hy.cfunc([A0DP, PERIGE, S, S4, PINVSQ, XI, ETA, ETASQ, EETA, PSISQ, COEF, COEF1, C1], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf([revday2radmin(16.05824518), np.deg2rad(72.8435), 0.0086731, .66816e-4, np.deg2rad(52.6988), np.deg2rad(110.5714), 0., np.deg2rad(115.9689)])

cmp_str="1.0401175226909520        3.1096479404901123E-002   1.2229280189271628E-002   1.0122292801892716       0.92448637336617012        35.857404780517960       0.32347120065050083       0.10463361765027657        2.8054980703618587E-003  0.89536638234972343        3.1084060647497392E-003   4.5765161876348661E-003   2.3338045281284853E-008"
cmp_arr = np.fromstring(cmp_str, sep=" ")

np.max(np.abs((cmp_arr - res)/cmp_arr))

8.925632787631372e-16

In [8]:
SINI0 = hy.sin(I0)
C3 = COEF*XI*A3OVK2*N0DP*SINI0/E0
X1MTH2 = 1.-THETA2
C4 = 2.*N0DP*COEF1*A0DP*BETA02*(ETA*(2.+.5*ETASQ)+E0*(.5+2.*ETASQ)-2.*CK2*XI/(A0DP*PSISQ)*(-3.*X3THM1*(1.-2.*EETA+ETASQ*(1.5-.5*EETA))+.75*X1MTH2*(2.*ETASQ-EETA*(1.+ETASQ))*hy.cos(2.*OMEGA0)))
C5 = 2.*COEF1*A0DP*BETA02*(1.+2.75*(ETASQ+EETA)+EETA*ETASQ)
THETA4 = THETA2**2
TEMP1 = 3.*CK2*PINVSQ*N0DP
TEMP2 = TEMP1*CK2*PINVSQ
TEMP3 = 1.25*CK4*PINVSQ**2*N0DP
MDOT = N0DP+.5*TEMP1*BETA0*X3THM1+.0625*TEMP2*BETA0*(13.-78.*THETA2+137.*THETA4)
OMGDOT = -.5*TEMP1*(1.-5.*THETA2)+0.0625*TEMP2*(7.-114.*THETA2+395.*THETA4)+TEMP3*(3.-36.*THETA2+49.*THETA4)
HDOT1 = -TEMP1*COSI0
N0DOT = HDOT1+(.5*TEMP2*(4.-19.*THETA2)+2.*TEMP3*(3.-7.*THETA2))*COSI0
OMGCOF = BSTAR*C3*hy.cos(OMEGA0)
MCOF = -TOTHRD*COEF*BSTAR/EETA
NODCF = 3.5*BETA02*HDOT1*C1
T2COF = 1.5*C1
LCOF = .125*A3OVK2*SINI0*(3.+5.*COSI0)/(1.+COSI0)
AYCOF = .25*A3OVK2*SINI0
DELM0 = (1.+ETA*hy.cos(M0))**3
SINM0 = hy.sin(M0)
X7THM1 = 7.*THETA2-1.

In [9]:
cf = hy.cfunc([SINI0, C3, X1MTH2, C4, C5, THETA4, TEMP1, TEMP2, TEMP3, MDOT, OMGDOT, HDOT1, N0DOT, OMGCOF, MCOF, NODCF, T2COF, LCOF, AYCOF, DELM0, SINM0, X7THM1], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf([revday2radmin(16.05824518), np.deg2rad(72.8435), 0.0086731, .66816e-4, np.deg2rad(52.6988), np.deg2rad(110.5714), 0., np.deg2rad(115.9689)])

cmp_str="0.95550259372444346        4.0375324468131845E-003  0.91298520661413884        3.7720114166826167E-004   1.2334919897178378E-002   7.5715742679841064E-003   1.0525006369286549E-004   5.2670486169768250E-008   4.6510490738020484E-008   7.0067293432081276E-002  -2.9717924905230478E-005  -3.1046947978737759E-005  -3.0963112435958149E-005   1.6348305656500835E-007  -4.9353389763819533E-005  -2.5358220065714951E-012   3.5007067921927278E-008   1.9357457580171985E-003   1.1203600999678345E-003  0.69630867249014317       0.93623504585812345      -0.39089644629897191"
cmp_arr = np.fromstring(cmp_str, sep=" ")

np.max(np.abs((cmp_arr - res)/cmp_arr))

5.748668562827581e-16

In [10]:
# For perigee less than 220 kilometers, the equations are
# truncated to linear variation in sqrt A and quadratic
# variation in mean anomaly.  Also, the C3 term, the
# delta OMEGA term, and the delta M term are dropped.
C1SQ = C1**2
D2 = 4.*A0DP*XI*C1SQ
TEMP0 = D2*XI*C1/3.
D3 = (17.*A0DP+S4)*TEMP0
D4 = .5*TEMP0*A0DP*XI*(221.*A0DP+31.*S4)*C1
T3COF = D2+2.*C1SQ
T4COF = .25*(3.*D3+C1*(12.*D2+10.*C1SQ))
T5COF = .2*(3.*D4+12.*C1*D3+6.*D2**2+15.*C1SQ*(2.*D2+C1SQ))

In [11]:
cf = hy.cfunc([C1SQ, D2, TEMP0, D3, D4, T3COF, T4COF, T5COF], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf([revday2radmin(12.), np.deg2rad(72.8435), 0.0086731, .66816e-4, np.deg2rad(52.6988), np.deg2rad(110.5714), 0., np.deg2rad(115.9689)])

cmp_str="4.4178125975612830E-024   8.8938566740110938E-023   2.4826571315925831E-034   5.5827388478537520E-033   4.0779833963340013E-043   9.7774191935233499E-023   4.7710772533442620E-033   2.8474903690952864E-043"
cmp_arr = np.fromstring(cmp_str, sep=" ")

np.max(np.abs((cmp_arr - res)/cmp_arr))

1.9532839596215615e-16

In [12]:
# Update for secular gravity and atmospheric drag.
MP = M0+MDOT*TSINCE
OMEGA = OMEGA0+OMGDOT*TSINCE
NODE = NODE0+(N0DOT+NODCF*TSINCE)*TSINCE
TEMPE = C4*TSINCE
TEMPA = 1.-C1*TSINCE
TEMPL = T2COF
TEMPF = MCOF*((1.+ETA*hy.cos(MP))**3-DELM0)+OMGCOF*TSINCE
# The conditional updates.
MP = MP + hy.select(hy.gte(PERIGE, SIMPHT), TEMPF, 0.)
OMEGA = OMEGA - hy.select(hy.gte(PERIGE, SIMPHT), TEMPF, 0.)
TEMPE = TEMPE + hy.select(hy.gte(PERIGE, SIMPHT), C5*(hy.sin(MP)-SINM0), 0.)
TEMPA = TEMPA - hy.select(hy.gte(PERIGE, SIMPHT),(D2+(D3+D4*TSINCE)*TSINCE)*TSINCE**2, 0.)
TEMPL = TEMPL + hy.select(hy.gte(PERIGE, SIMPHT),(T3COF+(T4COF+T5COF*TSINCE)*TSINCE)*TSINCE, 0.)
A = A0DP*TEMPA**2
N = KE/hy.sqrt(A**3)
E = E0-TEMPE*BSTAR
TEMPL = TEMPL*TSINCE**2

In [13]:
cf = hy.cfunc([MP, OMEGA, NODE, TEMPE, TEMPA, TEMPL, A, N, E], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf([revday2radmin(12.), np.deg2rad(72.8435), 0.0086731, .66816e-4, np.deg2rad(52.6988), np.deg2rad(110.5714), 30., np.deg2rad(115.9689)])

cmp_str="3.5006311214607546       0.91931591327868656        2.0235682034995306       -1.4019994318859061E-006  0.99999999993694422        2.8375100835128218E-009   1.2632189058319179        5.2379560119241088E-002   8.6731000936759936E-003"
cmp_arr = np.fromstring(cmp_str, sep=" ")

np.max(np.abs((cmp_arr - res)/cmp_arr))

1.4575818027208252e-16

In [14]:
# Long period periodics.
AXN = E*hy.cos(OMEGA)
AB = A*(1.-E**2)
AYN = AYCOF/AB+E*hy.sin(OMEGA)

In [15]:
cf = hy.cfunc([AXN, AB, AYN], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf([revday2radmin(12.), np.deg2rad(72.8435), 0.0086731, .66816e-4, np.deg2rad(52.6988), np.deg2rad(110.5714), 30., np.deg2rad(115.9689)])

cmp_str=" 5.2590580533442427E-003   1.2631238831390459        7.7837120758937391E-003"
cmp_arr = np.fromstring(cmp_str, sep=" ")

np.max(np.abs((cmp_arr - res)/cmp_arr))

0.0

In [16]:
# NOTE: this is the original Kepler-like solver, commented out
# because we can use the kepF() function from heyoka.py in its place.
# CAPU = FMOD2P(LCOF*AXN/AB+MP+OMEGA+N0DP*TEMPL)
# EPWNEW = CAPU
# for _ in range(10):
#     EPW = EPWNEW
#     SINEPW = hy.sin(EPW)
#     COSEPW = hy.cos(EPW)
#     ESINE = AXN*SINEPW-AYN*COSEPW
#     ECOSE = AXN*COSEPW+AYN*SINEPW
#     EPWNEW = (CAPU+ESINE-EPW)/(1.-ECOSE)+EPW

In [17]:
# Solve Kepler's equation.
CAPU = LCOF*AXN/AB+MP+OMEGA+N0DP*TEMPL
EPWNEW = hy.kepF(AYN, AXN, CAPU)
SINEPW = hy.sin(EPWNEW)
COSEPW = hy.cos(EPWNEW)
ESINE = AXN*SINEPW-AYN*COSEPW
ECOSE = AXN*COSEPW+AYN*SINEPW

In [18]:
cf = hy.cfunc([SINEPW, COSEPW, ESINE, ECOSE], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf([revday2radmin(12.), np.deg2rad(72.8435), 0.0086731, .66816e-4, np.deg2rad(52.6988), np.deg2rad(110.5714), 30., np.deg2rad(115.9689)])

cmp_str="-0.95674370609558179      -0.29093208975032464       -2.7670390722737840E-003  -8.9770462882492255E-003"
cmp_arr = np.fromstring(cmp_str, sep=" ")

np.max(np.abs((cmp_arr - res)/cmp_arr))

1.5673102463198297e-16

In [19]:
# Short period preliminary quantities
ELSQ = AXN**2+AYN**2
TEMPS = 1.-ELSQ
PL = A*TEMPS
R = A*(1.-ECOSE)
RDOT = KE*hy.sqrt(A)*ESINE/R
RFDOT = KE*hy.sqrt(PL)/R
BETAL = hy.sqrt(TEMPS)
TEMP3 = ESINE/(1.+BETAL)
COSU = (COSEPW-AXN+AYN*TEMP3)*A/R
SINU = (SINEPW-AYN-AXN*TEMP3)*A/R
U = ACTAN(SINU,COSU)
SIN2U = 2.*SINU*COSU
COS2U = 2.*COSU**2-1.
TEMP1 = CK2/PL
TEMP2 = TEMP1/PL

In [20]:
cf = hy.cfunc([ELSQ, TEMPS, PL, R, RDOT, RFDOT, BETAL, TEMP3, COSU, SINU, U, SIN2U, COS2U, TEMP1, TEMP2], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf([revday2radmin(12.), np.deg2rad(72.8435), 0.0086731, .66816e-4, np.deg2rad(52.6988), np.deg2rad(110.5714), 30., np.deg2rad(115.9689)])

cmp_str="8.8243865288858958E-005  0.99991175613471117        1.2631074345129614        1.2745588804217625       -1.8145731028600308E-004   6.5575259012531209E-002  0.99995587709394018       -1.3835500592615389E-003 -0.29356655639354601      -0.95593863661180434        4.4144333219460616       0.56126322734733747      -0.82763735393446991        4.2855261968173096E-004   3.3928437753750974E-004"
cmp_arr = np.fromstring(cmp_str, sep=" ")

np.max(np.abs((cmp_arr - res)/cmp_arr))

1.5672756691784474e-16

In [21]:
# Update for short periodics.
RK = R*(1.-1.5*TEMP2*BETAL*X3THM1)+.5*TEMP1*X1MTH2*COS2U
UK = U-.25*TEMP2*X7THM1*SIN2U
NODEK = NODE+1.5*TEMP2*COSI0*SIN2U
IK = I0+1.5*TEMP2*COSI0*SINI0*COS2U
RDOTK = RDOT-N*TEMP1*X1MTH2*SIN2U
RFDOTK = RFDOT+N*TEMP1*(X1MTH2*COS2U+1.5*X3THM1)

In [22]:
cf = hy.cfunc([RK, UK, NODEK, IK, RDOTK, RFDOTK], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf([revday2radmin(12.), np.deg2rad(72.8435), 0.0086731, .66816e-4, np.deg2rad(52.6988), np.deg2rad(110.5714), 30., np.deg2rad(115.9689)])

cmp_str="1.2748762763084123        4.4144519313380064        2.0236524628792609        1.2712401937490736       -1.9295991858536327E-004   6.5533415849018062E-002"
cmp_arr = np.fromstring(cmp_str, sep=" ")

np.max(np.abs((cmp_arr - res)/cmp_arr))

1.7416953240983777e-16

In [23]:
# Orientation vectors.
SINUK = hy.sin(UK)
COSUK = hy.cos(UK)
SINIK = hy.sin(IK)
COSIK = hy.cos(IK)
SINNOK = hy.sin(NODEK)
COSNOK = hy.cos(NODEK)
MX = -SINNOK*COSIK
MY = COSNOK*COSIK
UX = MX*SINUK+COSNOK*COSUK
UY = MY*SINUK+SINNOK*COSUK
UZ = SINIK*SINUK
VX = MX*COSUK-COSNOK*SINUK
VY = MY*COSUK-SINNOK*SINUK
VZ = SINIK*COSUK

In [24]:
cf = hy.cfunc([SINUK, COSUK, SINIK, COSIK, SINNOK, COSNOK, MX, MY, UX, UY, UZ, VX, VY, VZ], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0])

res = cf([revday2radmin(12.), np.deg2rad(72.8435), 0.0086731, .66816e-4, np.deg2rad(52.6988), np.deg2rad(110.5714), 30., np.deg2rad(115.9689)])

cmp_str="-0.95594409954138826      -0.29354876690595089       0.95546756666614729       0.29509613526624046       0.89920111058216312      -0.43753555595837523      -0.26535077255990763      -0.12911505160488243       0.38209852826648333      -0.14053260546818636      -0.91337358265767155      -0.34036614097546736       0.89748756014969366      -0.28047632601347694"
cmp_arr = np.fromstring(cmp_str, sep=" ")

np.max(np.abs((cmp_arr - res)/cmp_arr))

1.6309245999665671e-16

In [25]:
# Position and Velocity
PV1 = RK*UX
PV2 = RK*UY
PV3 = RK*UZ
PV4 = RDOTK*UX+RFDOTK*VX
PV5 = RDOTK*UY+RFDOTK*VY
PV6 = RDOTK*UZ+RFDOTK*VZ

In [26]:
sgp4 = hy.cfunc([PV1, PV2, PV3, PV4, PV5, PV6], [N0, I0, E0, BSTAR, OMEGA0, M0, TSINCE, NODE0], fast_math=True)

res = sgp4([revday2radmin(12.), np.deg2rad(72.8435), 0.0086731, .66816e-4, np.deg2rad(52.6988), np.deg2rad(110.5714), 30., np.deg2rad(115.9689)])

cmp_str="0.48712834889929890      -0.17916168475920063       -1.1644383119370860       -2.2379085558376696E-002   5.8842542658720215E-002  -1.8204327216298302E-002"
cmp_arr = np.fromstring(cmp_str, sep=" ")

np.max(np.abs((cmp_arr - res)/cmp_arr))

5.732008488078987e-15

In [27]:
res = sgp4([revday2radmin(16.05824518), np.deg2rad(72.8435), 0.0086731, .66816e-4, np.deg2rad(52.6988), np.deg2rad(110.5714), 0., np.deg2rad(115.9689)])
print(res[:3] * KMPER)
print(res[3:] * KMPER/60.)

[ 2328.969751931129  -5995.220511600548   1719.9729714023497]
[ 2.912073280385506  -0.9834179555026153 -7.090816207952735 ]


In [28]:
res = sgp4([revday2radmin(15.50103472202482), np.deg2rad(51.6439), 0.0007417, .38792e-4, np.deg2rad(17.6667), np.deg2rad(85.6398), 120., np.deg2rad(211.2001)])
print(res[:3] * KMPER)
print(res[3:] * KMPER/60.)

[ 4086.3396316729295  4796.928067515364  -2562.5856280208386]
[-5.287831886860389   1.6959008859460454 -5.265040553027974 ]
